In [ ]:
# default_exp annotation.yolo_adapter

In [ ]:
# hide
from nbdev.showdoc import *

In [ ]:
# export

import csv
import logging
import shutil
from os.path import join, splitext, basename, isfile
from mlcore.category_tools import read_categories
from mlcore.core import assign_arg_prefix
from mlcore.io.core import scan_files, create_folder
from mlcore.image.pillow_tools import get_image_size
from mlcore.annotation.core import Annotation, AnnotationAdapter, Region, RegionShape, SubsetType

In [ ]:
# hide
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
# export

FIELD_NAMES = ['class_number', 'c_x', 'c_y', 'width', 'height']
DEFAULT_IMAGES_FOLDER = 'images'
DEFAULT_IMAGE_ANNOTATIONS_FOLDER = 'labels'

In [ ]:
# export

logger = logging.getLogger(__name__)

# YOLO Annotation Adapter
> YOLO annotation adapter.

This adapter is tested with the [YOLOv3 Pytorch](https://github.com/ultralytics/yolov3) and [YOLOv5 Pytorch](https://github.com/ultralytics/yolov5) repositories.
Bounding boxes are normalized between [0,1].
Images should be in a separate folder (named e.g. *images/*).
Annotations should be in a separate folder (named e.g. *labels/*) and must have the same file name as corresponding image source file but with the ending *.txt*. The *categories.txt* file in the parent folder list all category labels which position (index) is used to label the bounding box class (see **Annotation Format** paragraph below). Below is an example structure:

```
categories.txt
images/
    image1.jpg
labels/
    image1.txt
```

Supported annotations:
- rectangle

## Bounding Box Format

The bounding box values are normalized between 0 and 1.
- Normalization Formula: ```x / image_width``` or ```y / image height``` 
- Normalization Formula: ```x * image_width``` or ```y * image height```

A bounding box has the following 4 parameters:
- Center X (bx)
- Center Y (by)
- Width (bw)
- Height (bh)

<img src="assets/yolo_bbox.png" alt="YOLO-BoundingBox" width="640" caption="YOLO bounding box parameters." />

Source: [A Gentle Introduction to YOLO v4 for Object detection in Ubuntu 20.04](https://robocademy.com/2020/05/01/a-gentle-introduction-to-yolo-v4-for-object-detection-in-ubuntu-20-04/)

## Annotation Format

Each annotation file (e.g. *image1.txt*) is a space separated CSV file where every row represents a bounding box in the image. 
A row has the following format: 
```<class_number> <center_x> <center_y> <width> <height>``` 
- *class_number*: The index of the class as listed in *categories.txt*
- *center_x*: The normalized bounding box center x value
- *center_y*: The normalized bounding box center y value
- *width*: The normalized bounding box width value
- *height*: The normalized bounding box height value

## Parameters

The adapter has the following parameters:
- `--path`: the path to the base folder containing the annotations (e.g.: *data/object_detection/my_collection*)
- `--categories_file_name`: tThe path to the categories file if not set, default to *categories.txt*
- `--images_folder_name`: the name of the folder containing the image files, if not set, default to *images*
- `--annotations_folder_name`: The name of the folder containing the image annotations, if not set, default to *labels*

In [ ]:
# export


class YOLOAnnotationAdapter(AnnotationAdapter):
    """
    Adapter to read and write annotations in the YOLO format.
    """

    def __init__(self, path, categories_file_name=None, images_folder_name=None, annotations_folder_name=None):
        """
        YOLO Adapter to read and write annotations.
        `path`: the folder containing the annotations
        `categories_file_name`: the name of the categories file
        `images_folder_name`: the name of the folder containing the image files
        `annotations_folder_name`: the name of the folder containing the mage annotations
        """
        super().__init__(path, categories_file_name)

        if images_folder_name is None:
            self.images_folder_name = DEFAULT_IMAGES_FOLDER
        else:
            self.images_folder_name = images_folder_name

        if annotations_folder_name is None:
            self.annotations_folder_name = DEFAULT_IMAGE_ANNOTATIONS_FOLDER
        else:
            self.annotations_folder_name = annotations_folder_name

    @classmethod
    def argparse(cls, prefix=None):
        """
        Returns the argument parser containing argument definition for command line use.
        `prefix`: a parameter prefix to set, if needed
        return: the argument parser
        """
        parser = super(YOLOAnnotationAdapter, cls).argparse(prefix=prefix)
        parser.add_argument(assign_arg_prefix('--images_folder_name', prefix),
                            dest="images_folder_name",
                            help="The name of the folder containing the image files.",
                            default=None)
        parser.add_argument(assign_arg_prefix('--annotations_folder_name', prefix),
                            dest="annotations_folder_name",
                            help="The name of the folder containing the mage annotations.",
                            default=None)
        return parser

    def read_annotations(self, subset_type=SubsetType.TRAINVAL):
        """
        Reads YOLO annotations.
        `subset_type`: the subset type to read
        return: the annotations as dictionary
        """
        path = join(self.path, str(subset_type))
        annotations = {}
        annotations_path = join(path, self.annotations_folder_name)
        images_path = join(path, self.images_folder_name)
        logger.info('Read images from {}'.format(images_path))
        logger.info('Read annotations from {}'.format(annotations_path))

        annotation_files = scan_files(annotations_path, file_extensions='.txt')
        categories = read_categories(join(self.path, self.categories_file_name))
        categories_len = len(categories)
        skipped_annotations = []

        for annotation_file in annotation_files:
            with open(annotation_file, newline='') as csv_file:
                annotation_file_name = basename(annotation_file)
                file_name, _ = splitext(annotation_file_name)
                file_path = join(images_path, '{}{}'.format(file_name, '.jpg'))

                if not isfile(file_path):
                    logger.warning("{}: Source file not found, skip annotation.".format(file_path))
                    skipped_annotations.append(file_path)
                    continue

                if annotation_file not in annotations:
                    annotations[annotation_file] = Annotation(annotation_id=annotation_file, file_path=file_path)

                annotation = annotations[annotation_file]

                reader = csv.DictReader(csv_file, fieldnames=FIELD_NAMES, delimiter=' ')
                _, image_width, image_height = get_image_size(file_path)
                for row in reader:
                    c_x = float(row["c_x"])
                    c_y = float(row["c_y"])
                    width = float(row["width"])
                    height = float(row["height"])
                    class_number = int(row["class_number"])
                    # denormalize bounding box
                    x_min = self._denormalize_value(c_x - (width / 2), image_width)
                    y_min = self._denormalize_value(c_y - (height / 2), image_height)
                    x_max = x_min + self._denormalize_value(width, image_width)
                    y_max = y_min + self._denormalize_value(height, image_height)
                    points_x = [x_min, x_max]
                    points_y = [y_min, y_max]

                    labels = [categories[class_number]] if class_number < categories_len else []
                    if not labels:
                        logger.warning("{}: Class number exceeds categories, set label as empty.".format(
                            annotation_file
                        ))
                    region = Region(shape=RegionShape.RECTANGLE, points_x=points_x, points_y=points_y, labels=labels)
                    annotation.regions.append(region)

        logger.info('Finished read annotations')
        logger.info('Annotations read: {}'.format(len(annotations)))
        if skipped_annotations:
            logger.info('Annotations skipped: {}'.format(len(skipped_annotations)))

        return annotations

    def write_annotations(self, annotations, subset_type=SubsetType.TRAINVAL):
        """
        Writes YOLO annotations to the annotations folder and copy the corresponding source files.
        `annotations`: the annotations as dictionary
        `subset_type`: the subset type to write
        return: a list of written target file paths
        """
        path = join(self.path, str(subset_type))
        create_folder(path)
        annotations_path = join(path, self.annotations_folder_name)
        annotations_folder = create_folder(annotations_path)
        images_path = join(path, self.images_folder_name)
        images_folder = create_folder(images_path)
        categories = read_categories(join(self.path, self.categories_file_name))

        logger.info('Write images to {}'.format(images_folder))
        logger.info('Write annotations to {}'.format(annotations_folder))

        copied_files = []
        skipped_annotations = []

        for annotation in annotations.values():
            annotation_file_name = basename(annotation.file_path)
            file_name, _ = splitext(annotation_file_name)
            annotations_file = join(annotations_folder, '{}{}'.format(file_name, '.txt'))
            target_file = join(images_folder, annotation_file_name)

            if not isfile(annotation.file_path):
                logger.warning("{}: Source file not found, skip annotation.".format(annotation.file_path))
                skipped_annotations.append(annotation.file_path)
                continue
            if isfile(target_file):
                logger.warning("{}: Target file already exist, skip annotation.".format(annotation.file_path))
                skipped_annotations.append(annotation.file_path)
                continue

            _, image_width, image_height = get_image_size(annotation.file_path)
            rows = []
            skipped_regions = []
            for index, region in enumerate(annotation.regions):
                if region.shape != RegionShape.RECTANGLE:
                    logger.warning('Unsupported shape {}, skip region {} at path: {}'.format(region.shape,
                                                                                             index,
                                                                                             annotations_file))
                    skipped_regions.append(region)
                    continue

                x_min, x_max = region.points_x
                y_min, y_max = region.points_y
                width = x_max - x_min
                height = y_max - y_min
                # normalize bounding box
                c_x = self._normalize_value(x_min + width / 2, image_width)
                c_y = self._normalize_value(y_min + height / 2, image_height)
                width = self._normalize_value(width, image_width)
                height = self._normalize_value(height, image_height)
                label = region.labels[0] if len(region.labels) else ''
                try:
                    class_number = categories.index(label)
                except ValueError:
                    logger.warning('Unsupported label {}, skip region {} at path: {}'.format(label,
                                                                                             index,
                                                                                             annotations_file))
                    skipped_regions.append(region)
                    continue
                rows.append(dict(zip(FIELD_NAMES, [class_number, c_x, c_y, width, height])))

            if len(skipped_regions) == len(annotation.regions):
                logger.warning("{}: All regions skipped, skip annotation.".format(annotation.file_path))
                skipped_annotations.append(annotation.file_path)
                continue

            # write the annotations
            with open(annotations_file, 'w', newline='') as csv_file:
                writer = csv.DictWriter(csv_file, fieldnames=FIELD_NAMES, delimiter=' ')
                writer.writerows(rows)

            # copy the file
            shutil.copy2(annotation.file_path, target_file)
            copied_files.append(target_file)

        logger.info('Finished write annotations')
        logger.info('Annotations written: {}'.format(len(annotations) - len(skipped_annotations)))
        if skipped_annotations:
            logger.info('Annotations skipped: {}'.format(len(skipped_annotations)))
        return copied_files

    @classmethod
    def _denormalize_value(cls, value, metric):
        """
        Denormalize a bounding box value
        `value`: the value to denormalize
        `metric`: the metric to denormalize from
        return: the denormalized value
        """
        return int(value * metric)

    @classmethod
    def _normalize_value(cls, value, metric):
        """
        Normalize a bounding box value
        `value`: the value to normalize
        `metric`: the metric to normalize against
        return: the normalized value
        """
        return float(value) / metric


In [ ]:
show_doc(YOLOAnnotationAdapter.list_files)
show_doc(YOLOAnnotationAdapter.read_annotations)
show_doc(YOLOAnnotationAdapter.read_categories)
show_doc(YOLOAnnotationAdapter.write_files)
show_doc(YOLOAnnotationAdapter.write_annotations)
show_doc(YOLOAnnotationAdapter.write_categories)
show_doc(YOLOAnnotationAdapter.argparse)

In [ ]:
# hide

# for generating scripts from notebook directly
from nbdev.export import notebook2script
notebook2script()

Converted annotation-core.ipynb.
Converted annotation-folder_category_adapter.ipynb.
Converted annotation-multi_category_adapter.ipynb.
Converted annotation-via_adapter.ipynb.
Converted annotation-yolo_adapter.ipynb.
Converted annotation_converter.ipynb.
Converted annotation_viewer.ipynb.
Converted category_tools.ipynb.
Converted core.ipynb.
Converted dataset-core.ipynb.
Converted dataset-image_classification.ipynb.
Converted dataset-image_object_detection.ipynb.
Converted dataset-image_segmentation.ipynb.
Converted dataset-type.ipynb.
Converted dataset_generator.ipynb.
Converted evaluation-core.ipynb.
Converted geometry.ipynb.
Converted image-color_palette.ipynb.
Converted image-inference.ipynb.
Converted image-opencv_tools.ipynb.
Converted image-pillow_tools.ipynb.
Converted image-tools.ipynb.
Converted index.ipynb.
Converted io-core.ipynb.
Converted tensorflow-tflite_converter.ipynb.
Converted tensorflow-tflite_metadata.ipynb.
Converted tensorflow-tfrecord_builder.ipynb.
Converted t